***Reinforcement Learning using Qlearning and Enhanced Qlearning in a 2 Player Nim's Game***

In [1]:
import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game()->list:
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(_st:list)->(int,int):
    xored = _st[0] ^ _st[1] ^ _st[2] #so _st is a list of the piles and here we are taking the XOR of them, so this expression only evaluates to 1 if one of the piles has a different num than the other 
    if xored == 0:
        return nim_random(_st) #if all are the same make a random choice
    for pile in range(3): #else look at the piles
        s = _st[pile] ^ xored # look at the pile and evaluate if it has a different number than 1, if it does s is 1, if it doesn't s is 0
        if s <= _st[pile]: # this will hold true if the pile isn't empty.
            return _st[pile]-s, pile #set move to 1 less than the pile count from the pile that doesn't have 1 piece left. EX: If a pile has 8 pieces set move to 7

# Random Nim player
def nim_random(_st:list)->(int,int):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [2]:
def nim_qlearner(_st:list)->(int,int):
    global qtable
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    return move, pile  # action

In [3]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(_a:str, _b:str):
    state, side = init_game(), 'A'  #init_game starts the board which is a list of ints referring to the pile
    while True:
        engine = Engines[_a] if side == 'A' else Engines[_b] #if it's A turn set engine to the corresponding player
        move, pile = engine(state)   #call engine on the state of the board, this calls each individual engine
        #print(state, move, pile)  # debug purposes
        state[pile] -= move  #conduct the move on the pile
        if state == [0, 0, 0]:  # game ends
            return side  # winning side is the side that performed the last clear above
        side = 'B' if side == 'A' else 'A'  # switch sides, this is performed at the end of a turn.

def play_games(_n:int, _a:str, _b:str)->(int,int): #creates a number _n of games to simulate
    from collections import defaultdict
    wins = defaultdict(int)
    for _ in range(_n):
        wins[game(_a, _b)] += 1  #call a game with a and b
    # info
    print(f"{_n} games, {_a:>8s}{wins['A']:5d}  {_b:>8s}{wins['B']:5d}")
    return wins['A'], wins['B']

In [4]:
# Play games
#play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
#play_games(1000, 'Random', 'Guru')
#play_games(1000, 'Guru', 'Guru') ;

1000 games,     Guru  996    Random    4


(996, 4)

In [5]:
qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n:int):
    global qtable
    Reward = 100.0
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=np.float32) #30 item array in 11 in 11 in 11
    # play _n games
    for _ in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1) #random pick
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game

            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            
            # Switch sides for play and learning
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r:float, _st1:list, move:int, pile:int, q_future_best:float):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [6]:
%%time

nim_qlearn(1000)

CPU times: user 50.7 ms, sys: 0 ns, total: 50.7 ms
Wall time: 50.2 ms


In [7]:
# Play games
play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Random', 'Qlearner')

play_games(1000, 'Random', 'Random') ;

1000 games, Qlearner  702    Random  298
1000 games,   Random  297  Qlearner  703
1000 games,   Random  501    Random  499


In [8]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
Wins = []
for n in n_train:
    nim_qlearn(n)
    print("Train :", n)
    wins_a, wins_b = play_games(1000, 'Qlearner', 'Random')
    Wins += [wins_a/(wins_a+wins_b)]

Train : 3
1000 games, Qlearner  487    Random  513
Train : 10
1000 games, Qlearner  562    Random  438
Train : 100
1000 games, Qlearner  658    Random  342
Train : 1000
1000 games, Qlearner  694    Random  306
Train : 10000
1000 games, Qlearner  717    Random  283
Train : 50000
1000 games, Qlearner  740    Random  260
Train : 100000
1000 games, Qlearner  699    Random  301
CPU times: user 7.24 s, sys: 16.4 ms, total: 7.26 s
Wall time: 7.26 s


In [9]:
# Check the ratio of wins wrt to size of the reinforcement model training
print(Wins)

[0.487, 0.562, 0.658, 0.694, 0.717, 0.74, 0.699]


In [10]:
# Function to print the entire set of states
def qtable_log(_fn:str):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r
            print(s, file=fout)

qtable_log('qtable_debug.txt')

***Understanding Reinforcement Learning and the Nim's Game***

***What is the environment in the Nim model?***

The environment in the Nim model is the state of the current board. That is, it's the number of items in each of the piles, by which the agent can impose an action on said environment mainly through removing items from each pile.

***What is the agent(s) in the Nim learning model (Hint, not just the Q-learner). Is
Guru an agent?***

The agent is the player, a game consists of two agents and these players are different algorithms used to act on the environment. Guru is an agent, it's an algorithm that uses a Nim solution to calculate it's next step (the amount of items to take from a pile and which pile to take from). 

Qlearner is another agent. This one stores the expected value of states and actions in a Q-table, then it uses this table to indicate to the agent the expected reward for taking an action in this particular state. The values of the Q-table are then updated.

Random is another agent. This one decides it's action through a random number generator.

***What is the reward and penalty in the Nim learning model.***

A state in the environment where the player has achieved a winning condition. The reward here is a win, and it's indicated by the current player taking the final item (to clear the last pile).
The penalty is being the player that does not win, not taking the item that clears the last pile.

***How many possible states there could be in the Nim game with a maximum of 10
items per pile and 3 piles total?***

Consider if there was 3 items per pile, and 2 piles total. There would be in the first pile 4 different item numbers 3, 2, 1, and 0. This is combined with the same number of states in the second pile, thus the number of states would be 16 which is 4^2.

Now consider if there was 10 items per pile and 3 piles total, the number of states would be 11^3 = 1331 possible states.

***How many possible unique actions there could be for player 1 take for their first
action in a Nim game with 10 items per pile and 3 piles total?***


A player can take any amount of chips from one of the piles in its first move. The number of chips they can take is up to the total number of items in any given pile. So they can take up to 10 items which is 10 actions, these actions can occur on one of any of the piles, so 3*10 = ***30 possible initial actions.***

***Now we will improve the provided Nim game learning model, by implementing a system of Rewards and Punishments, as well as a system that shuffles player turns during training doing away with the heavy first turn bias. Finally for a good Q learner we must train our Q learner to balance between random learning and exploitation (learning from the Q table itself), otherwise it will not perform well***


In [11]:
ITEMS_MX = 10

# Random Nim player
def nim_random(_st:list)->(int,int):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

def init_game()->list:
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

In [12]:
import random
qtable_NEW, Alpha, Gamma, Reward, epsilon = None, 1, 0.8, [100.0,-100.0], 0.2

def nim_qlearner_NEW(_st:list)->(int,int):
    global qtable_NEW
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable_NEW[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    return move, pile  # action


# learn from _n games, randomly played to explore the possible states
def nim_qlearn_NEW(_n:int):
    global qtable_NEW
    # based on max items per pile
    qtable_NEW = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=np.float32)
    # play _n games
    Reward = [100.0, -100.0]
    side = ['A','B']
    for _ in range(_n):
        side = random.choice(side)
        # first state is starting position
        st1 = init_game()
        count = 0
        move = None
        pile = None
        while True:  # while game not finished
            # make a random move - exploration
            #print('side',side) - debug
            #Keep track of last move, pile, and state (coming from previous loop)
            lastmove = move
            lastpile = pile
            if random.random()<epsilon or np.argmax(qtable_NEW[st1[0], st1[1], st1[2]]) == 0:
                move, pile = nim_random(st1)
                #print('r')
                #print(st1, pile, move)
            else:
                a = np.argmax(qtable_NEW[st1[0], st1[1], st1[2]])  # exploitation
                move, pile = a%ITEMS_MX+1, a//ITEMS_MX
                if move <= 0 or st1[pile] < move:
                    move, pile = nim_random(st1) 
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            #print("st2", st2, "move", move)
            if st2 == [0, 0, 0] and side=='A':  # game ends
                qtable_update_NEW(Reward[0], st1, move, pile, 0)  # Reward the one who won
                #print(Reward[0], st1, move, pile, 0)
                #print(Reward[1], lastState, lastmove, lastpile, 0)  
                break  # new game
            elif st2 == [0, 0, 0] and side=='B':
                qtable_update_NEW(Reward[1], lastState, lastmove, lastpile, 0)# punish the move conducted on the previous state that lead to loss to that player
                break
            if side=='A':
                qtable_update_NEW(0, st1, move, pile, np.max(qtable_NEW[st2[0], st2[1], st2[2]]))
            #print(0, st1, move, pile, np.max(qtable_NEW[st2[0], st2[1], st2[2]]))
            
            
            #print(lastmove, lastpile)
            # Switch sides for play and learning
            lastState = st1 #keep track of the last state
            st1 = st2
            side = 'B' if side == 'A' else 'A'
            
            
# Equation 3 - update the qtable
def qtable_update_NEW(r:float, _st1:list, move:int, pile:int, q_future_best:float):
    a = pile*ITEMS_MX+move-1
    #print(_st1[0], _st1[1], _st1[2], qtable[_st1[0], _st1[1], _st1[2], a], 'move', move, 'pile', pile,'best future:', q_future_best, 'reward', r)
    qtable_NEW[_st1[0], _st1[1], _st1[2], a] =(1-Alpha)* qtable_NEW[_st1[0], _st1[1], _st1[2], a]+  Alpha * (r + Gamma * q_future_best)
    

This algorithm took me a while to create, I tested different methods with two explicit sides, shuffling the player and what not, ultimately the algorithm behaved similarly, and this was the cleanest implementation. Basically, the sides here are implicitly defined, all that matters is a reward associated to the winning move, and a punishment associated to the losing move. This way the Q-table doesn't have to learn just from one side, it could continuously improve and know which game states to avoid and which game states to prioritize via the reward.

In [13]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner_NEW':nim_qlearner_NEW, 'Qlearner':nim_qlearner}

def game(_a:str, _b:str):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[_a] if side == 'A' else Engines[_b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n:int, _a:str, _b:str)->(int,int):
    from collections import defaultdict
    wins = defaultdict(int)
    for _ in range(_n):
        wins[game(_a, _b)] += 1
    # info
    print(f"{_n} games, {_a:>8s}{wins['A']:5d}  {_b:>8s}{wins['B']:5d}")
    return wins['A'], wins['B']


In [14]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
Wins = []
for n in n_train:
    nim_qlearn_NEW(n)
    print("Train ", n)
    wins_a, wins_b = play_games(1000, 'Qlearner_NEW', 'Random')
    Wins += [wins_a/(wins_a+wins_b)]

Train  3
1000 games, Qlearner_NEW  501    Random  499
Train  10
1000 games, Qlearner_NEW  525    Random  475
Train  100
1000 games, Qlearner_NEW  666    Random  334
Train  1000
1000 games, Qlearner_NEW  783    Random  217
Train  10000
1000 games, Qlearner_NEW  803    Random  197
Train  50000
1000 games, Qlearner_NEW  813    Random  187
Train  100000
1000 games, Qlearner_NEW  796    Random  204
CPU times: user 9.34 s, sys: 13 ms, total: 9.36 s
Wall time: 9.33 s


In [15]:
# Function to print the entire set of states
def qtable_log(_fn:str):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable_NEW[i, j, k, a]
                s += ',%.1f' % r
            print(s, file=fout)

qtable_log('qtableNEW_debug.txt')

In [16]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
Wins = []
for n in n_train:
    nim_qlearn(n)
    wins_a, wins_b = play_games(1000, 'Qlearner', 'Random')
    Wins += [wins_a/(wins_a+wins_b)]

1000 games, Qlearner  479    Random  521
1000 games, Qlearner  505    Random  495
1000 games, Qlearner  643    Random  357
1000 games, Qlearner  728    Random  272
1000 games, Qlearner  728    Random  272
1000 games, Qlearner  709    Random  291
1000 games, Qlearner  727    Random  273
CPU times: user 7.24 s, sys: 12.1 ms, total: 7.25 s
Wall time: 7.24 s


In comparison to the Random learner, Qlearner_NEW overcomes Qlearner when faced off against Random. With increased training size, the Qlearner_NEW is able to achieve a higher win rate, acquiring increased learning capability from the presence of a penalty. 

In [17]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
Wins = []
for n in n_train:
    nim_qlearn(n)
    nim_qlearn_NEW(n)
    wins_a, wins_b = play_games(1000, 'Qlearner', 'Qlearner_NEW')
    Wins += [wins_a/(wins_a+wins_b)]

1000 games, Qlearner  529  Qlearner_NEW  471
1000 games, Qlearner  505  Qlearner_NEW  495
1000 games, Qlearner  510  Qlearner_NEW  490
1000 games, Qlearner  302  Qlearner_NEW  698
1000 games, Qlearner  256  Qlearner_NEW  744
1000 games, Qlearner  297  Qlearner_NEW  703
1000 games, Qlearner  318  Qlearner_NEW  682
CPU times: user 16.5 s, sys: 8.15 ms, total: 16.5 s
Wall time: 16.5 s


In [19]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
Wins = []
for n in n_train:
    nim_qlearn(n)
    nim_qlearn_NEW(n)
    wins_a, wins_b = play_games(1000, 'Qlearner_NEW', 'Qlearner')
    Wins += [wins_a/(wins_a+wins_b)]

1000 games, Qlearner_NEW  532  Qlearner  468
1000 games, Qlearner_NEW  453  Qlearner  547
1000 games, Qlearner_NEW  475  Qlearner  525
1000 games, Qlearner_NEW  718  Qlearner  282
1000 games, Qlearner_NEW  792  Qlearner  208
1000 games, Qlearner_NEW  731  Qlearner  269
1000 games, Qlearner_NEW  760  Qlearner  240
CPU times: user 15.7 s, sys: 384 ms, total: 16.1 s
Wall time: 16.1 s


When faced off against Q_learner, Qlearner_NEW beats Qlearner when learned on a larger sample size, this indicates that the capacity for learning information for the new algorithm is greater than the Qlearner. Qlearner when trained on samples of 100, and 1000, is able to comfortably overcome the first turn bias which is present all throughout the increased training sizes. This indicates that the addition of the penalty gives more information per training turn.

There is a heavy first turn bias when both algorithms are pitted against each other and I'm unsure why this is. I've tried with this implementation to not give the computer the first turn, in hopes that the first turn bias can be remedied. What helped is actually inserting a exploitation/exploration method in my training protocol, while maintaining proper sides shuffling.

In [20]:
play_games(1000, 'Qlearner_NEW', 'Qlearner')
play_games(1000, 'Qlearner', 'Qlearner_NEW')

1000 games, Qlearner_NEW  717  Qlearner  283
1000 games, Qlearner  255  Qlearner_NEW  745


(255, 745)

In [28]:
play_games(1000, 'Qlearner_NEW', 'Random')
play_games(1000, 'Random', 'Qlearner_NEW')

play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Random', 'Qlearner')

1000 games, Qlearner_NEW  794    Random  206
1000 games,   Random  233  Qlearner_NEW  767
1000 games, Qlearner  717    Random  283
1000 games,   Random  271  Qlearner  729


(271, 729)

In [47]:
play_games(1000, 'Qlearner_NEW', 'Guru')
play_games(1000, 'Guru', 'Qlearner_NEW')

1000 games, Qlearner_NEW   10      Guru  990
1000 games,     Guru  997  Qlearner_NEW    3


(997, 3)

In [44]:
play_games(1000, 'Qlearner', 'Guru')
play_games(1000, 'Guru', 'Qlearner')

1000 games, Qlearner   24      Guru  976
1000 games,     Guru  997  Qlearner    3


(997, 3)

Q_learner_New has no improvement when compared against Guru. Guru exploits the board, basically taking the maximal amount of items in a pile save 1, then doing this across all piles, by the time it comes to the last piles they move then the opponent moves and then they clear the board. To be able to do this we need to take the best action every state action that we get, and currently the q-learning doesn't implement a method to solve the Nim game. 

Essentially, the Q_learner learning against both Random and itself, both pale in comparison to a solver like Guru. 